In [1]:
!pip install tinytag sounddevice

You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%%bash

#wget https://os.unil.cloud.switch.ch/fma/fma_small.zip
unzip fma_small.zip

Archive:  fma_small.zip


replace fma_small/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [3]:
import os
from tinytag import TinyTag, TinyTagException
from sklearn.neighbors import NearestNeighbors
from collections import defaultdict
from tensorflow.keras.models import load_model
import librosa
from collections import Counter
import multiprocessing
from tqdm import tqdm
from tensorflow.keras.models import Model
import numpy as np
import sounddevice as sd

In [4]:
MUSIC_ROOT = 'fma_small/'
mp3s = []
for root, subdirs, files in os.walk(MUSIC_ROOT):
    for fn in files:
        if fn.endswith('.mp3'):
            mp3s.append(os.path.join(root, fn))
len(mp3s)

8000

In [5]:
TO_SKIP = {'Podcast', 'Books & Spoken'}

def process_mp3(path):
    try:
        tag = TinyTag.get(path)
        if tag.genre in TO_SKIP:
            return None
    except TinyTagException:
        print('error')
        return None
    # signal, sr = librosa.load(path, res_type='kaiser_fast', offset=20, duration=30)
    signal, sr = librosa.load(path, res_type='kaiser_fast', offset=0, duration=30)
    try:
        melspec = librosa.feature.melspectrogram(signal, sr=sr).T[:1280,]
        if len(melspec) != 1280:
            return None
    except ValueError:
        return None
    return {'path': path,
            'melspecs': np.asarray(np.split(melspec, 10)),
            'tag': tag}

songs = [process_mp3(path) for path in tqdm(mp3s[:1000])]
songs = [song for song in songs if song]

100%|██████████| 1000/1000 [11:38<00:00,  1.43it/s]


In [6]:
inputs = []
for song in songs:
    inputs.extend(song['melspecs'])
inputs = np.array(inputs)
inputs.shape

(10000, 128, 128)

In [7]:
inputs

array([[[2.65136933e+01, 1.86410770e+01, 3.00668853e+01, ...,
         1.30287272e-02, 1.12126724e-02, 1.05849529e-02],
        [1.42122369e+01, 4.70750045e+01, 2.84955517e+01, ...,
         3.62580816e-03, 2.81737498e-03, 2.64885249e-03],
        [9.43835423e+00, 4.95771451e+01, 1.66709201e+01, ...,
         3.56512210e-04, 1.22964362e-05, 3.17170364e-06],
        ...,
        [9.78877376e+00, 4.55389706e+01, 2.57436138e+01, ...,
         1.18182603e-03, 1.52246150e-04, 1.90809914e-05],
        [3.98039588e+00, 6.97215854e+01, 5.67598878e+01, ...,
         8.35636365e-04, 1.09446446e-04, 1.31287421e-05],
        [1.57657281e+00, 1.09572076e+02, 7.55950115e+01, ...,
         3.95235622e-04, 6.40336355e-05, 6.13213521e-06]],

       [[2.15457655e+00, 1.79887708e+02, 1.25011621e+02, ...,
         2.66460670e-04, 4.16075876e-05, 4.32625530e-06],
        [1.81736989e+00, 2.33842012e+02, 1.54576938e+02, ...,
         1.97825094e-04, 3.60279270e-05, 3.30803931e-06],
        [3.36705892e+00, 

In [8]:
signal, sr = librosa.load(mp3s[0], offset=30, duration=30) #, res_type='kaiser_fast', offset=30, duration=30)
signal

array([], dtype=float32)

In [9]:
cnn_model = load_model('zoo/15/song_classify.h5')
vectorize_model = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-4].output)
vectors = vectorize_model.predict(inputs)
vectors.shape

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


(10000, 256)

In [10]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(vectors)

def most_similar_songs(song_idx):
    distances, indices = nbrs.kneighbors(vectors[song_idx * 10: song_idx * 10 + 10])
    c = Counter()
    for row in indices:
        for idx in row[1:]:
            c[idx // 10] += 1
    return c.most_common()

In [11]:
song_idx = 7
print(songs[song_idx]['path'])

print('---')
for idx, score in most_similar_songs(song_idx)[:5]:
    print(songs[idx]['path'], score)
print('')

fma_small/137/137463.mp3
---
fma_small/137/137463.mp3 54
fma_small/011/011772.mp3 6
fma_small/043/043608.mp3 5
fma_small/011/011771.mp3 4
fma_small/011/011770.mp3 4



In [12]:
duration = 30  # seconds
fs = 22050
myrecording = sd.rec(int(duration * fs), samplerate=fs, channels=1)

In [13]:
myrecording.shape

(661500, 1)

In [14]:
sd.play(myrecording, samplerate=fs)

In [15]:
myrecording.min()

/home/ohtamans/.pyenv/versions/3.6.7/envs/cookbook/lib/python3.6/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)


nan

In [16]:
signal, sr = librosa.load('/Users/douwe/Dropbox/Apps/Hi-Q Recordings/recording-20180219-162112.mp3', res_type='kaiser_fast', offset=0, duration=30)

melspec = librosa.feature.melspectrogram(signal, sr=sr).T[:1280,]
melspecs = np.asarray(np.split(melspec, 10))
melspecs.shape

FileNotFoundError: [Errno 2] No such file or directory: '/Users/douwe/Dropbox/Apps/Hi-Q Recordings/recording-20180219-162112.mp3'

In [ ]:
recorded_vectors = vectorize_model.predict(melspecs)

In [ ]:
distances, indices = nbrs.kneighbors(recorded_vectors)
c = Counter()
for row in indices:
    for idx in row[1:]:
        c[idx // 10] += 1
for idx, _ in c.most_common():
    print(songs[idx]['path'])


In [ ]:
signal, sr = librosa.load('/Users/douwe/Dropbox/Apps/Hi-Q Recordings/recording-20180219-162112.mp3', res_type='kaiser_fast', offset=0, duration=30)

In [ ]:
sd.play(signal.flatten(), samplerate=sr)